In [4]:
import sentence_transformers

/Users/dhruv.nigam/PycharmProjects/whuphf/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
# infer embeddings from text using bge en model
model = sentence_transformers.SentenceTransformer('BAAI/bge-large-en-v1.5')


def infer_embeddings(texts, model):
    embeddings = model.encode(texts)
    return embeddings


text = "This is a test sentence"
infer_embeddings([text], model)

array([[ 0.0218855 ,  0.00638117,  0.00838451, ...,  0.00260959,
        -0.01417644, -0.01027322]], dtype=float32)

In [6]:
import pandas as pd

df_dilogue = pd.read_csv('../data/the-office-lines - scripts.csv')
df_dilogue['word_count'] = df_dilogue['line_text'].apply(lambda x: len(x.split()))

n = 3
df_dilogue['context'] = ''

for index, row in df_dilogue.iterrows():
    context_lines = []
    # Get past n rows
    for i in range(max(0, index - n), index):
        context_lines.append(df_dilogue.loc[i, 'speaker'] + " : " + df_dilogue.loc[i, 'line_text'])
    # Add current row with '>>'
    context_lines.append(df_dilogue.loc[index, 'speaker'] + " : >> " + df_dilogue.loc[index, 'line_text'])
    # Get future n rows
    for i in range(index + 1, min(df_dilogue.shape[0], index + n + 1)):
        context_lines.append(df_dilogue.loc[i, 'speaker'] + " : " + df_dilogue.loc[i, 'line_text'])

    # Join all context lines with a newline character
    df_dilogue.at[index, 'context'] = "\n".join(context_lines)

df_dilogue

,id,season,episode,scene,line_text,speaker,deleted,word_count,context
0,1,1,1,1,All right Jim. Your quarterlies look very good...,Michael,False,14,Michael : >> All right Jim. Your quarterlies l...
1,2,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,False,9,Michael : All right Jim. Your quarterlies look...
2,3,1,1,1,So you've come to the master for guidance? Is ...,Michael,False,14,Michael : All right Jim. Your quarterlies look...
3,4,1,1,1,"Actually, you called me in here, but yeah.",Jim,False,8,Michael : All right Jim. Your quarterlies look...
4,5,1,1,1,"All right. Well, let me show you how it's done.",Michael,False,10,"Jim : Oh, I told you. I couldn't close it. So...."
...,...,...,...,...,...,...,...,...,...
59904,59905,9,23,112,It all seems so very arbitrary. I applied for ...,Creed,False,59,Creed : [still playing and singing] And all th...
59905,59906,9,23,113,I just feel lucky that I got a chance to share...,Meredith,False,41,Erin : How did you do it? How did you capture ...
59906,59907,9,23,114,I���m happy that this was all filmed so I can ...,Phyllis,False,31,"Darryl : Everyday when I came into work, all I..."
59907,59908,9,23,115,I sold paper at this company for 12 years. My ...,Jim,False,46,Creed : It all seems so very arbitrary. I appl...


In [11]:
# for each row in the df , infer embeddings for the context and create a new dataframe with only id and the embeddings

df_dilogue['context_embeddings'] = df_dilogue['context'].apply(lambda x: infer_embeddings([x], model)[0])
df_dilogue_embeddings = df_dilogue[['id', 'context_embeddings']]
df_dilogue_embeddings.to_pickle('../data/the-office-lines-embeddings.pkl')
